In [ ]:
# 单因素分析
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

# 设置中文显示（如果需要）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 设置输出目录
output_dir = r'results\2_LOG'
# 确保输出目录存在
os.makedirs(output_dir, exist_ok=True)

# 1. 读取数据
data = pd.read_excel('data.xlsx')

# 2. 数据预处理
# 定义因变量和自变量
dependent_var = '1yearegfr'
categorical_vars = ['Gender', 'K-W_nodules', 'Crescent-shaped_changes', 'Capillary_microaneurysms', 'Hyaline_droplets', 
                    'Cap_lesion', 'Tubular_atrophy', 'Interstitial_fibrosis', 'Arteriolar_hyalinosis', 'Mesangial_expansion' ]
continuous_vars = ['Age', 'BMI', 'AIP', 'TyG', 'SII', 'ePWV', '24h-UP', 'eGFR']

# 检查因变量是否为二分类
if len(data[dependent_var].unique()) != 2:
    raise ValueError("因变量必须是二分类变量")

# 对分类变量进行编码（如果尚未编码）
le = LabelEncoder()
for var in categorical_vars:
    if data[var].dtype == 'object':
        data[var] = le.fit_transform(data[var])

# 3. 单因素Logistic回归分析
results = []
significant_vars = []  # 用于存储p≤0.05的变量名

for var in categorical_vars + continuous_vars:
    # 添加常数项
    X = sm.add_constant(data[var])
    y = data[dependent_var]
    
    # 拟合模型（自动处理缺失值）
    try:
        model = sm.Logit(y, X, missing='drop').fit(disp=0)
        
        # 获取结果
        coef = model.params[1]
        p_value = model.pvalues[1]
        odds_ratio = np.exp(coef)
        ci = np.exp(model.conf_int().iloc[1, :])
        
        # 添加p值判断：若p>0.05，输出"否"，反之输出"是"
        p_value_significant = "否" if p_value > 0.05 else "是"
        
        # 如果p≤0.05，记录变量名
        if p_value <= 0.05:
            significant_vars.append(var)
        
        results.append({
            'Variable': var,
            'OR': odds_ratio,
            'Lower CI': ci[0],
            'Upper CI': ci[1],
            'P-value': p_value,
            'P>0.05': p_value_significant
        })
    except:
        print(f"无法计算变量 {var} 的回归结果，可能由于数据问题")
        results.append({
            'Variable': var,
            'OR': np.nan,
            'Lower CI': np.nan,
            'Upper CI': np.nan,
            'P-value': np.nan,
            'P>0.05': np.nan
        })

# 将结果转换为DataFrame
results_df = pd.DataFrame(results)

# 添加一行显示所有p≤0.05的变量名
if significant_vars:
    significant_vars_str = ', '.join(significant_vars)
    summary_row = pd.DataFrame({
        'Variable': ['显著变量 (p≤0.05):'],
        'OR': [significant_vars_str],
        'Lower CI': [''],
        'Upper CI': [''],
        'P-value': [''],
        'P>0.05': ['']
    })
    results_df = pd.concat([results_df, summary_row], ignore_index=True)
else:
    summary_row = pd.DataFrame({
        'Variable': ['显著变量 (p≤0.05):'],
        'OR': ['无'],
        'Lower CI': [''],
        'Upper CI': [''],
        'P-value': [''],
        'P>0.05': ['']
    })
    results_df = pd.concat([results_df, summary_row], ignore_index=True)

print("分析完成，结果已保存到指定目录")
print("\n回归结果汇总:")
print(results_df.to_string())

# 将结果保存到Excel - 修改路径
output_excel_path = os.path.join(output_dir, 'logistic_regression_results.xlsx')
results_df.to_excel(output_excel_path, index=False)
print(f"\n回归结果已保存为 {output_excel_path}")

In [ ]:
#以Age作为调整变量

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import os

# 设置输出路径
output_dir = r'results\2_LOG'
os.makedirs(output_dir, exist_ok=True)  # 确保目录存在

# 设置中文显示（如果需要）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 1. 读取数据
data = pd.read_excel('data.xlsx')

# 2. 数据预处理
# 定义因变量和自变量
dependent_var = '1yearegfr'
adjustment_var = 'Age'  # 调整变量
categorical_vars = ['Gender', 'K-W_nodules', 'Crescent-shaped_changes', 'Capillary_microaneurysms', 'Hyaline_droplets', 'Cap_lesion', 'Tubular_atrophy', 'Interstitial_fibrosis', 'Arteriolar_hyalinosis', 'Mesangial_expansion' ]
continuous_vars = ['BMI', 'AIP', 'TyG', 'SII', 'ePWV', '24h-UP', 'eGFR']

# 检查因变量是否为二分类
if len(data[dependent_var].unique()) != 2:
    raise ValueError("因变量必须是二分类变量")

# 对分类变量进行编码（如果尚未编码）
le = LabelEncoder()
for var in categorical_vars:
    if data[var].dtype == 'object':
        data[var] = le.fit_transform(data[var])

# 3. 单因素Logistic回归分析（调整AGE）
results = []
significant_vars = []  # 存储p≤0.05的变量名

for var in categorical_vars + continuous_vars:
    # 准备自变量：包括当前变量和调整变量AGE
    X = data[[var, adjustment_var]]
    X = sm.add_constant(X)  # 添加常数项
    y = data[dependent_var]
    
    # 拟合模型（自动处理缺失值）
    try:
        model = sm.Logit(y, X, missing='drop').fit(disp=0)
        
        # 获取当前变量的结果（不包括AGE的结果）
        coef = model.params[1]  # 第一个系数是当前变量
        p_value = model.pvalues[1]
        odds_ratio = np.exp(coef)
        ci = np.exp(model.conf_int().iloc[1, :])
        
        # 检查p值是否≤0.05
        if p_value <= 0.05:
            significant_vars.append(var)
        
        results.append({
            'Variable': var,
            'OR': odds_ratio,
            'Lower CI': ci[0],
            'Upper CI': ci[1],
            'P-value': p_value,
            'Adjustment': 'AGE'  # 记录调整了哪些变量
        })
    except Exception as e:
        print(f"无法计算变量 {var} 的回归结果，错误: {str(e)}")
        results.append({
            'Variable': var,
            'OR': np.nan,
            'Lower CI': np.nan,
            'Upper CI': np.nan,
            'P-value': np.nan,
            'Adjustment': 'AGE'
        })

# 将结果转换为DataFrame
results_df = pd.DataFrame(results)

print("\n回归结果汇总(调整AGE):")
print(results_df.to_string())

# 打印所有p≤0.05的变量名
print("\nP值≤0.05的变量:")
if significant_vars:
    for var in significant_vars:
        print(f"- {var}")
else:
    print("没有P值≤0.05的变量")

# 将结果保存到Excel到指定路径
excel_path = os.path.join(output_dir, 'AGE.xlsx')
results_df.to_excel(excel_path, index=False)
print(f"\n回归结果已保存为 {excel_path}")

In [ ]:
#以eGFR为调整变量

import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import os

# 设置输出路径
output_path = r'results\2_LOG'
# 确保输出目录存在
os.makedirs(output_path, exist_ok=True)

# 设置中文显示（如果需要）
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

# 1. 读取数据
data = pd.read_excel('data.xlsx')

# 2. 数据预处理
# 定义因变量和自变量
dependent_var = '1yearegfr'
adjustment_var = 'eGFR'  # 调整变量
categorical_vars = ['Gender', 'K-W_nodules', 'Crescent-shaped_changes', 'Capillary_microaneurysms', 'Hyaline_droplets', 'Cap_lesion', 'Tubular_atrophy', 'Interstitial_fibrosis', 'Arteriolar_hyalinosis', 'Mesangial_expansion' ]
continuous_vars = ['Age', 'BMI', 'AIP', 'TyG', 'SII', 'ePWV', '24h-UP']

# 检查因变量是否为二分类
if len(data[dependent_var].unique()) != 2:
    raise ValueError("因变量必须是二分类变量")

# 对分类变量进行编码（如果尚未编码）
le = LabelEncoder()
for var in categorical_vars:
    if data[var].dtype == 'object':
        data[var] = le.fit_transform(data[var])

# 3. 共线性检查函数
def check_collinearity(X, var_names):
    """
    计算方差膨胀因子(VIF)检查共线性
    VIF > 5 表示中度共线性，>10 表示严重共线性
    """
    vif_data = pd.DataFrame()
    vif_data["Variable"] = var_names
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(var_names))]
    return vif_data

# 4. 单因素Logistic回归分析（调整iEGFR）
results = []
collinearity_results = []
significant_vars = []  # 存储p≤0.05的变量名

for var in categorical_vars + continuous_vars:
    # 准备自变量：包括当前变量和调整变量iEGFR
    X = data[[var, adjustment_var]].dropna()  # 去除缺失值
    y = data[dependent_var].loc[X.index]  # 对齐因变量
    
    # 检查共线性
    vif_results = check_collinearity(sm.add_constant(X), ['const', var, adjustment_var])
    collinearity_results.append({
        'Variable': var,
        'VIF_current_var': vif_results.loc[1, 'VIF'],
        'VIF_iEGFR': vif_results.loc[2, 'VIF']
    })
    
    # 如果当前变量与iEGFR存在严重共线性(VIF>10)
    if vif_results.loc[1, 'VIF'] > 10 or vif_results.loc[2, 'VIF'] > 10:
        print(f"警告: 变量 {var} 与 iEGFR 存在严重共线性 (VIF = {max(vif_results.loc[1, 'VIF'], vif_results.loc[2, 'VIF']):.2f})")
        results.append({
            'Variable': var,
            'OR': np.nan,
            'Lower CI': np.nan,
            'Upper CI': np.nan,
            'P-value': np.nan,
            'Adjustment': 'iEGFR',
            'VIF_current_var': vif_results.loc[1, 'VIF'],
            'VIF_iEGFR': vif_results.loc[2, 'VIF'],
            'Note': 'Severe collinearity'
        })
        continue
    
    # 添加常数项
    X = sm.add_constant(X)
    
    # 拟合模型
    try:
        model = sm.Logit(y, X).fit(disp=0)
        
        # 获取当前变量的结果（不包括iEGFR的结果）
        coef = model.params[1]  # 第一个系数是当前变量
        p_value = model.pvalues[1]
        odds_ratio = np.exp(coef)
        ci = np.exp(model.conf_int().iloc[1, :])
        
        # 检查p值是否≤0.05
        if p_value <= 0.05:
            significant_vars.append(var)
        
        results.append({
            'Variable': var,
            'OR': odds_ratio,
            'Lower CI': ci[0],
            'Upper CI': ci[1],
            'P-value': p_value,
            'Adjustment': 'iEGFR',
            'VIF_current_var': vif_results.loc[1, 'VIF'],
            'VIF_iEGFR': vif_results.loc[2, 'VIF'],
            'Note': ''
        })
    except Exception as e:
        print(f"无法计算变量 {var} 的回归结果，错误: {str(e)}")
        results.append({
            'Variable': var,
            'OR': np.nan,
            'Lower CI': np.nan,
            'Upper CI': np.nan,
            'P-value': np.nan,
            'Adjustment': 'iEGFR',
            'VIF_current_var': vif_results.loc[1, 'VIF'],
            'VIF_iEGFR': vif_results.loc[2, 'VIF'],
            'Note': str(e)
        })

# 将结果转换为DataFrame
results_df = pd.DataFrame(results)
collinearity_df = pd.DataFrame(collinearity_results)

# 6. 输出结果
print("分析完成，结果已保存到指定路径")
print("\n回归结果汇总(调整iEGFR):")
print(results_df.to_string())

print("\n共线性检查结果:")
print(collinearity_df.to_string())

# 打印所有p≤0.05的变量名
print("\n所有p≤0.05的变量名:")
if significant_vars:
    for var in significant_vars:
        print(var)
else:
    print("没有p≤0.05的变量")

# 保存结果到Excel
output_file = os.path.join(output_path, 'EGFR.xlsx')
with pd.ExcelWriter(output_file) as writer:
    results_df.to_excel(writer, sheet_name='Regression Results', index=False)
    collinearity_df.to_excel(writer, sheet_name='Collinearity Check', index=False)
    
    # 将显著变量列表也保存到Excel
    significant_df = pd.DataFrame({'Significant Variables (p≤0.05)': significant_vars})
    significant_df.to_excel(writer, sheet_name='Significant Variables', index=False)

print(f"\n回归结果和共线性检查已保存为 {output_file}")